In [ ]:
%pip install lonboard overturemaps

In [ ]:

import overturemaps as om
from overturemaps import core
import overturemaps
import pandas
import geopandas as gpd
from shapely import wkb
from lonboard import Map, PolygonLayer, ScatterplotLayer
import ipywidgets as widgets
import numpy as np
from IPython.display import display

'''
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from huggingface_hub import HfApi, hf_hub_download
'''

In [ ]:
bbox = 139.6917, 35.6895, 139.7017, 35.6995
bbox = -109.048, 37.000, -102.040, 41.000 #colorado

In [ ]:
def create_map(dataset):
    layer = ScatterplotLayer.from_geopandas(
        dataset,
        get_fill_color=[0, 128, 128],
        radius_min_pixels = 1.5,
    )

    view_state = {
        "longitude": (bbox[0]+bbox[2])/2,
        "latitude": (bbox[1]+bbox[3])/2,
        "zoom": 8,
        "pitch": 0,
    }
    return Map(layer,view_state=view_state)
    

In [ ]:
gdf = core.geodataframe("place", bbox=bbox)
print(np.shape(gdf))

In [ ]:
create_map(gdf)

In [ ]:
# Randomly sample 10,000 points from gdf
sampled_gdf = gdf.sample(n=10000, random_state=42)
print(sampled_gdf.shape)

In [ ]:
create_map(sampled_gdf)

In [ ]:
def dist_from_address(point, address):
    """
    Calculate the distance from a point to an address.
    """
    # Convert the address to a point
    address_point = gdf[gdf['address'] == address].geometry.values[0]
    # Calculate the distance
    distance = point.distance(address_point)
    return distance

In [ ]:
def OVERTURE_address_to_coord(address):
    """
    Convert an address to coordinates.
    """
    # Get the coordinates of the address
    coords = gdf[gdf['address'] == address].geometry.values[0]
    return coords.x, coords.y

In [ ]:
def CROSSREFERENCED_address_to_coord(address):
    """
    #TODO find a way to cross reference address with other dataset. 
    """

In [ ]:
def check_on_road(address):
   print(f'address: {address}')
   bbox = address.x-0.0001, address.y+-.0001, address.x+0.0001, address.y+0.0001
   print(f"BBOX: {bbox}")
   roads = overturemaps.record_batch_reader("connector", bbox).read_all()
   print("Len roads: ", len(roads))
   if len(roads) > 0:
      return True
   else:
      return False
   #gdf = core.geodataframe("transportation", bbox=(0,0,0,0))

In [ ]:
for i in range(len(sampled_gdf)):
    point_coords = sampled_gdf.geometry.values[i]
    #print(f'Point {i}\'s coordinates: {point_coords}')
    
    #print(f'Point {i}\'s address: {addess}')
    #print(f'Point {i}\'s coordinates: {sampled_gdf.geometry.values[i]}')
    print(f"Is address {i} on a road? {check_on_road(point_coords)}")
    # Calculate the point from the point to the address
    #distance = dist_from_address(point, "Tokyo Tower")
    #print(f"Distance from {point} to Tokyo Tower: {distance} meters")

### What the following code does:

Gather place dataset of a region

Gather address dataset of a region

Pair them up

Find Distance

In [ ]:
bbox = 35.6895,139.6917,   35.6995, 139.7017
bbox = -104.548, 39.000, -103.540, 40.000 #colorado

In [46]:
places_dataset = core.geodataframe("place",bbox=bbox)
print(places_dataset.shape)
address_dataset = core.geodataframe("address",bbox=bbox)
print(address_dataset.shape)

# build uppercase address strings
address_list = (address_dataset['number'] + ' ' + address_dataset['street']).str.upper().tolist()

# pair each address string with its (lon, lat) tuple from the geometry column
address_dict = {
    addr.upper(): (pt.x, pt.y)
    for addr, pt in zip(address_list, address_dataset.geometry)
}


# extract the “freeform” string from each place’s addresses and uppercase it
places_list = places_dataset['addresses'] \
    .apply(lambda lst: lst[0].get('freeform', '')) \
    .str.upper() \
    .tolist()

# now build a dict mapping each freeform address to its (lon, lat)
places_dict = {
    addr.upper(): (pt.x, pt.y)
    for addr, pt in zip(places_list, places_dataset.geometry) if addr != None
}


print(len(address_dict),len(address_list))
print(len(places_dict),len(places_list))

(1282, 15)
(13621, 12)
12035 13621
932 1282


In [ ]:
for address in address_list:
    if address not in address_dict.keys():
        print(f"Address {address} not found in address list")

In [50]:
print(places_dict)

{'17355 MCKENZE LANE': (-104.509045, 39.0051084), '17466 BLUE LAKE LN': (-104.5050026, 39.0022391), '12080 BRADSHAW RD': (-104.49773, 39.0064), '11975 BRADSHAW RD': (-104.4923537, 39.0039644), '17925 E US HIGHWAY 24': (-104.4945576, 39.0154144), '18005 U S HIGHWAY 24': (-104.4882777, 39.0173157), '18045 E US HIGHWAY 24': (-104.4896362, 39.0185236), '12990 N. PEYTON HWY ': (-104.48078, 39.0188786), '12505 PEYTON HWY': (-104.4764126, 39.0122602), '19407 ELLIOTT VW': (-104.467158, 39.0172376), '19784 ELLIOTT VW': (-104.4598912, 39.019956), '21150 SAMPSON RD': (-104.4348357, 39.0064813), '13375 N LOG RD': (-104.4215996, 39.0233536), '21875 E US HIGHWAY 24': (-104.418412, 39.023204), '23486 SCOTT RD': (-104.38862, 39.0000182), '12065 N ELLICOTT HWY': (-104.3862485, 39.0052673), '12475 N ELLICOTT HWY': (-104.3858837, 39.0109946), '12240 E RANGE VIEW DR': (-104.3788396, 39.0080022), '13370 PALOMINO DR': (-104.3776284, 39.0227392), '24536 PERCHERON WAY': (-104.3738092, 39.0210559), '11494 N CA

In [ ]:

create_map(address_dataset)

In [ ]:
create_map(places_dataset)

In [ ]:
def find_matching_address(point_address):
    if point_address in address_dict.keys():
        return address_dict[point_address]


In [56]:
def get_dist(point,address):
    print(f"Point: {point} Address: {address} !!")
    dist = np.sqrt((point[0] - address[0])**2 + (point[1] - address[1])**2)
    print(f"Distance between {point} and {address}: ", dist)
    return dist

In [61]:
distances = {}
for point,val in zip(places_dict.keys(),places_dict.values()):
    print(f'point: {point} {val}')
    if point in address_dict.keys():
        distance = get_dist(address_dict[point],places_dict[point])
        distances[point] = distance
    else:
        continue

point: 17355 MCKENZE LANE (-104.509045, 39.0051084)
point: 17466 BLUE LAKE LN (-104.5050026, 39.0022391)
point: 12080 BRADSHAW RD (-104.49773, 39.0064)
point: 11975 BRADSHAW RD (-104.4923537, 39.0039644)
point: 17925 E US HIGHWAY 24 (-104.4945576, 39.0154144)
point: 18005 U S HIGHWAY 24 (-104.4882777, 39.0173157)
point: 18045 E US HIGHWAY 24 (-104.4896362, 39.0185236)
point: 12990 N. PEYTON HWY  (-104.48078, 39.0188786)
point: 12505 PEYTON HWY (-104.4764126, 39.0122602)
point: 19407 ELLIOTT VW (-104.467158, 39.0172376)
point: 19784 ELLIOTT VW (-104.4598912, 39.019956)
point: 21150 SAMPSON RD (-104.4348357, 39.0064813)
point: 13375 N LOG RD (-104.4215996, 39.0233536)
point: 21875 E US HIGHWAY 24 (-104.418412, 39.023204)
point: 23486 SCOTT RD (-104.38862, 39.0000182)
point: 12065 N ELLICOTT HWY (-104.3862485, 39.0052673)
point: 12475 N ELLICOTT HWY (-104.3858837, 39.0109946)
point: 12240 E RANGE VIEW DR (-104.3788396, 39.0080022)
point: 13370 PALOMINO DR (-104.3776284, 39.0227392)
point:

In [62]:
print(distances.values())

dict_values([0.00011876840670037284, 0.0004880866962288836, 0.00037391737867382817, 0.00026021174744261756, 0.00028963892821781283, 0.0012790709711838644, 0.0016979015719734661, 0.00016456827926217063, 0.002287165197371621, 0.001423095494450851, 0.1291629023194709, 0.00035225598797602157, 0.003921264385141671, 0.0010383500055081813, 9.001907712038463e-05, 1.2521360704121319e-05, 0.00019263681386348327, 0.0004292231238888277, 2.8621041453471877e-05, 0.0005725731666142485])
